In [ ]:

# # ExCIR — Cross-Domain Benchmarks (Anonymous, IEEE Reproducibility)
#
# This notebook reproduces the ExCIR experiments across **29 datasets**, comparing against
# common global attribution baselines. It is self-contained and robust:
#
# - Deterministic seeding, single-run reproducibility
# - XGBoost GPU acceleration when available, graceful CPU fallback
# - OpenML/text loaders with try/except → synthetic fallback
# - Clean CSV outputs in `out/` + paper-style figures
#
# **Methods compared**: ExCIR (ours), TreeGain, PFI, PDP-var, MI(pred), MI(label), Surrogate-LR
# **Figures**: AOPC curves, stability histograms, Pareto cost-agreement, agreement bundle
# **Tables**: base model metrics, feature_importances, AOPC summary/curves, top-k sufficiency, stability, lightweight checks

# %%
# ============ 0) Environment & Determinism ============
import os, sys, csv, time, warnings, random
from dataclasses import dataclass
from typing import Dict, List, Tuple

warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import gaussian_kde, entropy
from scipy.signal import periodogram

# GPU-capable model (falls back to CPU automatically in our code)
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, r2_score
from sklearn.inspection import permutation_importance, partial_dependence
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import check_random_state

from sklearn.feature_selection import mutual_info_classif, mutual_info_regression

# Safe imports for datasets (avoid recursive naming)
from sklearn.datasets import (
    fetch_openml,
    fetch_20newsgroups,
    fetch_california_housing,
    load_diabetes as skl_load_diabetes,
    load_digits as skl_load_digits,
    load_iris as skl_load_iris,
    load_wine as skl_load_wine,
    load_breast_cancer as skl_load_breast_cancer,
    make_classification,
    make_regression,
)

sys.setrecursionlimit(3000)

def set_seed(seed: int = 0, threads: int = 1):
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ["OMP_NUM_THREADS"] = str(threads)
    random.seed(seed)
    np.random.seed(seed)

set_seed(0, threads=1)
print("Determinism set.")

# %%
# ============ 1) ExCIR core (CIR) & BlockCIR ============

def _center_midmean(X: np.ndarray, y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Robust centering via mid-mean (average of Q1, Q3)."""
    q1x = np.quantile(X, 0.25, axis=0)
    q3x = np.quantile(X, 0.75, axis=0)
    Xc = X - 0.5 * (q1x + q3x)
    q1y, q3y = np.quantile(y, [0.25, 0.75])
    yc = y - 0.5 * (q1y + q3y)
    return Xc, yc

def _center_median(X: np.ndarray, y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    return X - np.median(X, axis=0), y - np.median(y)

def _center_mean(X: np.ndarray, y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    return X - X.mean(axis=0), y - y.mean()

def center_cols(X, y, how="midmean"):
    if how == "midmean": return _center_midmean(X, y)
    if how == "median":  return _center_median(X, y)
    if how == "mean":    return _center_mean(X, y)
    raise ValueError(f"Unknown centering: {how}")

def excir_feature_scores(X: np.ndarray, preds: np.ndarray, centering="midmean") -> np.ndarray:
    """ExCIR per-feature score: 0.5*(1 + sum(x_tilde*y_tilde) / sum(|x_tilde*y_tilde|))."""
    Xc, yc = center_cols(np.asarray(X, float), np.asarray(preds, float).ravel(), centering)
    P = Xc * yc[:, None]
    N = P.sum(axis=0)
    D = np.abs(P).sum(axis=0)
    out = 0.5 * (1.0 + np.divide(N, D, out=np.zeros_like(N), where=D>0))
    out[D == 0] = 0.5
    return out

def blockcir_scores(X: np.ndarray, preds: np.ndarray, blocks: Dict[str, List[int]], centering="midmean") -> Dict[str, float]:
    """Group-wise ExCIR over correlated feature blocks."""
    Xc, yc = center_cols(np.asarray(X, float), np.asarray(preds, float).ravel(), centering)
    P = Xc * yc[:, None]
    scores = {}
    for name, idxs in blocks.items():
        idxs = np.asarray(list(idxs), dtype=int)
        pG = P[:, idxs].sum(axis=1)
        uG = np.abs(P[:, idxs]).sum(axis=1)
        NG, DG = pG.sum(), uG.sum()
        scores[name] = 0.5 * (1.0 + (NG / DG)) if DG > 0 else 0.5
    return scores

def expand_block_to_features(block_scores: Dict[str, float], d: int, blocks: Dict[str, List[int]]) -> np.ndarray:
    vals = np.zeros(d)
    for name, idxs in blocks.items():
        for j in idxs:
            vals[j] = block_scores[name]
    return vals

# %%
# ============ 2) Agreement Metrics & AOPC ============

def jaccard_at_k(a_idx, b_idx, k: int):
    A, B = set(a_idx[:k]), set(b_idx[:k])
    u = len(A | B)
    return (len(A & B) / u) if u else 1.0

def precision_at_k(ref_idx, cand_idx, k: int):
    A, B = set(ref_idx[:k]), set(cand_idx[:k])
    return len(A & B) / k

def spearman_rank_corr(a: np.ndarray, b: np.ndarray):
    """Fast Spearman using rank-of-rank trick."""
    ra = a.argsort().argsort().astype(float)
    rb = b.argsort().argsort().astype(float)
    ra = (ra - ra.mean()) / (ra.std() if ra.std() else 1.0)
    rb = (rb - rb.mean()) / (rb.std() if rb.std() else 1.0)
    return float((ra * rb).mean())

def projection_alignment_residual(y_full, y_light):
    """Affine residual between y_full and y_light after best affine fit."""
    y = np.asarray(y_full).ravel(); yp = np.asarray(y_light).ravel()
    n = y.shape[0]
    Phi = np.c_[yp, np.ones(n)]
    theta, *_ = np.linalg.lstsq(Phi, y, rcond=None)
    yhat = Phi @ theta
    return float(np.linalg.norm(y - yhat) / (np.linalg.norm(y) + 1e-12))

def kde_kl_symmetric(y_full, y_light, grid_points=400):
    """Symmetric KL between KDEs of y_full and y_light."""
    y = np.asarray(y_full).ravel(); yp = np.asarray(y_light).ravel()
    kde_p = gaussian_kde(y); kde_q = gaussian_kde(yp)
    both = np.r_[y, yp]; lo, hi = np.percentile(both, 0.5), np.percentile(both, 99.5)
    xs = np.linspace(lo, hi, grid_points)
    p = np.clip(kde_p(xs), 1e-12, None); q = np.clip(kde_q(xs), 1e-12, None)
    dx = (hi - lo)/max(grid_points-1, 1)
    kl_pq = float(np.sum(p*(np.log(p)-np.log(q))) * dx)
    kl_qp = float(np.sum(q*(np.log(q)-np.log(p))) * dx)
    return 0.5*(kl_pq + kl_qp)

def rank_from_scores(scores):
    return np.argsort(-scores)

def aopc_curves(task, model, Xte, yte, scores, steps=9):
    """Insertion↑ / Deletion↓ curves by masking/unmasking top-k features."""
    idx = rank_from_scores(scores)
    n, d = Xte.shape
    fracs = np.linspace(0, 1, steps)
    ins, dele = [], []
    for f in fracs:
        k = max(1, int(f*d))
        keep = idx[:k]
        mask = np.zeros(d, bool); mask[keep] = True
        X_ins = np.where(mask, Xte, 0.0).astype(np.float32)  # reveal top-k
        X_del = np.where(mask, 0.0, Xte).astype(np.float32)  # remove top-k
        y_ins = model.predict(X_ins); y_del = model.predict(X_del)
        if task == "clf":
            ins.append(accuracy_score(yte, y_ins)); dele.append(accuracy_score(yte, y_del))
        else:
            ins.append(r2_score(yte, y_ins)); dele.append(r2_score(yte, y_del))
    return fracs, np.asarray(ins), np.asarray(dele)

def stability_noise_eval(scores_fn, Xva, p_va, repeats=15, sigma=0.01):
    base = scores_fn(Xva, p_va)
    cirs, j8 = [], []
    for _ in range(repeats):
        noise = np.random.normal(0, sigma, size=Xva.shape)
        s_noisy = scores_fn(Xva + noise, p_va)
        cirs.append(spearman_rank_corr(base, s_noisy))
        j8.append(jaccard_at_k(base.argsort()[::-1], s_noisy.argsort()[::-1], k=min(8, Xva.shape[1])))
    return np.asarray(cirs), np.asarray(j8)

# %%
# ============ 3) Baselines (PFI, TreeGain, PDP-var, MI, Surrogate-LR) ============

def pfi_scores(model, X, y, n_repeats=5, seed=0):
    try:
        return permutation_importance(model, X.astype(np.float32), y,
                                      n_repeats=n_repeats, random_state=seed,
                                      scoring=None).importances_mean
    except Exception:
        return np.zeros(X.shape[1])

def tree_gain_scores(model, d):
    try:
        w = getattr(model, "feature_importances_", None)
        return w if (w is not None and len(w)==d) else np.zeros(d)
    except Exception:
        return np.zeros(d)

def pdp_var_scores(model, X, grid_points=15, random_state=0):
    d = X.shape[1]; scores = np.zeros(d)
    rs = check_random_state(random_state)
    idx = rs.choice(np.arange(X.shape[0]), size=min(1000, X.shape[0]), replace=False)
    Xs = X[idx]
    for j in range(d):
        try:
            pd = partial_dependence(model, Xs.astype(np.float32), features=[j],
                                    kind='average', grid_resolution=grid_points)
            scores[j] = np.var(pd.average[0])
        except Exception:
            scores[j] = 0.0
    return scores

def surrogate_lr_scores(X, teacher_scores, alpha=1.0):
    X = np.asarray(X, float); y = np.asarray(teacher_scores, float).ravel()
    reg = Ridge(alpha=alpha, random_state=0).fit(X, y)
    w = np.abs(reg.coef_); w = np.linalg.norm(w, axis=0) if w.ndim>1 else w
    s = w/(w.sum()+1e-12)
    return s

def mi_pred_scores(X, preds, task):
    X = np.asarray(X, float); p = np.asarray(preds, float).ravel()
    if task=="clf":
        z = (p > np.median(p)).astype(int);
        return mutual_info_classif(X, z, random_state=0)
    z = np.digitize(p, np.quantile(p, [1/3, 2/3]))
    return mutual_info_classif(X, z, random_state=0)

def mi_label_scores(X, y, task):
    return mutual_info_classif(X, y, random_state=0) if task=="clf" else mutual_info_regression(X, y, random_state=0)

# %%
# ============ 4) XGBoost Trainer with GPU → CPU Fallback ============

def _get_xgb_model(task: str, y_data: np.ndarray, params: dict):
    unique_classes = np.unique(y_data)
    if task == "clf":
        if len(unique_classes) > 2:
            return xgb.XGBClassifier(objective='multi:softmax', num_class=len(unique_classes),
                                     eval_metric='merror', **params)
        else:
            return xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss',
                                     use_label_encoder=False, **params)
    else:
        return xgb.XGBRegressor(objective='reg:squarederror', eval_metric='rmse', **params)

def train_gboost(task, Xtr, ytr, Xva, yva, seed=0):
    Xall = np.vstack([Xtr, Xva]).astype(np.float32)
    yall = np.r_[ytr, yva]
    params = dict(n_estimators=50, random_state=seed, tree_method='gpu_hist',
                  gpu_id=0, validate_parameters=True, n_jobs=1)
    m = _get_xgb_model(task, yall, params)
    try:
        m.fit(Xall, yall); return m
    except xgb.core.XGBoostError:
        print("  [XGBoost] GPU failed. Falling back to CPU 'hist'.")
        params['tree_method'] = 'hist'; params.pop('gpu_id', None)
        m = _get_xgb_model(task, yall, params)
        try:
            m.fit(Xall, yall); return m
        except Exception as e_cpu:
            print(f"  [XGBoost] CPU fallback failed: {e_cpu}")
            return None
    except Exception as e:
        print(f"  [XGBoost] Training failed: {e}")
        return None

def predict_scores(task, model, X):
    if model is None: return np.zeros(X.shape[0])
    X = X.astype(np.float32)
    try:
        if task=="clf":
            # robustly get probabilities when available; else predictions
            if hasattr(model, "predict_proba"):
                proba = model.predict_proba(X)
                if proba.ndim == 2:
                    if proba.shape[1] == 2:
                        return proba[:, 1]
                    return proba[:, -1]
            pred = model.predict(X)
            # if predict() returns class labels, coerce to [0,1] for ExCIR (best-effort)
            if pred.ndim == 1 and np.issubdtype(pred.dtype, np.integer):
                return (pred == pred.max()).astype(float)
            return pred
        return model.predict(X)
    except Exception:
        return np.zeros(X.shape[0])

# %%
# ============ 5) Dataset Plumbing & 29 Loaders (with fallbacks) ============

@dataclass
class DatasetPack:
    name: str
    task: str  # "clf" | "reg"
    Xtr: np.ndarray; ytr: np.ndarray
    Xva: np.ndarray; yva: np.ndarray
    Xte: np.ndarray; yte: np.ndarray
    feature_names: List[str]

def _standardize_fit(Xtr, Xva, Xte):
    ss = StandardScaler(with_mean=True, with_std=True)
    return ss.fit_transform(Xtr), ss.transform(Xva), ss.transform(Xte)

def _split_xy(X, y, task, seed=0):
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2,
                                          stratify=y if task=="clf" else None,
                                          random_state=seed)
    Xtr, Xva, ytr, yva = train_test_split(Xtr, ytr, test_size=0.25,
                                          stratify=ytr if task=="clf" else None,
                                          random_state=seed)
    Xtr_s, Xva_s, Xte_s = _standardize_fit(Xtr, Xva, Xte)
    feat_names = [f"f{i}" for i in range(X.shape[1])]
    return Xtr_s, ytr, Xva_s, yva, Xte_s, yte, feat_names

def _split_pack_from_xy(X, y, name, task, seed):
    Xtr, ytr, Xva, yva, Xte, yte, feat = _split_xy(X, y, task, seed)
    return DatasetPack(name, task, Xtr, ytr, Xva, yva, Xte, yte, feat)

# --- Textish / OpenMLish with fallbacks ---
def load_20ng_binary(seed=0):
    try:
        cats=['comp.graphics','sci.space']
        tr = fetch_20newsgroups(subset='train', categories=cats, remove=('headers','footers','quotes'))
        te = fetch_20newsgroups(subset='test',  categories=cats, remove=('headers','footers','quotes'))
        tf = TfidfVectorizer(max_features=3000, ngram_range=(1,2), stop_words='english')
        Xtr_full = tf.fit_transform(tr.data).astype(np.float32).toarray()
        Xte = tf.transform(te.data).astype(np.float32).toarray()
        ytr_full, yte = tr.target, te.target
        Xtr, Xva, ytr, yva = train_test_split(Xtr_full, ytr_full, test_size=0.2,
                                              stratify=ytr_full, random_state=seed)
        feat = [f"tfidf_{i}" for i in range(Xtr.shape[1])]
        return DatasetPack("20ng_bin","clf",Xtr,ytr,Xva,yva,Xte,yte,feat)
    except Exception:
        return load_synthetic_clf(seed)

def load_adult(seed=0):
    try:
        ds = fetch_openml("adult", version=2, as_frame=True)
        dfX = ds.data.select_dtypes(include=[np.number]).copy()
        y = (ds.target == '>50K').astype(int).to_numpy()
        X = dfX.to_numpy(float)
        Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed)
        Xtr, Xva, ytr, yva = train_test_split(Xtr, ytr, test_size=0.2, stratify=ytr, random_state=seed)
        Xtr, Xva, Xte = _standardize_fit(Xtr, Xva, Xte)
        return DatasetPack("adult","clf",Xtr,ytr,Xva,yva,Xte,yte,list(dfX.columns))
    except Exception:
        return load_synthetic_clf(seed)

def load_har6(seed=0):
    try:
        ds = fetch_openml("HAR", version=1, as_frame=True)
        X = ds.data.to_numpy(float); y = ds.target.astype('category').cat.codes.to_numpy()
        return _split_pack_from_xy(X, y, "har6", "clf", seed)
    except Exception:
        return load_synthetic_clf(seed)

def load_digits8x8(seed=0):
    dg = skl_load_digits()
    X = dg.data.astype(float)/16.0; y = dg.target
    return _split_pack_from_xy(X, y, "digits8x8", "clf", seed)

def load_california(seed=0):
    try:
        ds = fetch_california_housing()
        X = ds.data.astype(float); y = ds.target.astype(float)
        return _split_pack_from_xy(X, y, "california", "reg", seed)
    except Exception:
        return load_synthetic_reg(seed)

def load_diabetes(seed=0):
    ds = skl_load_diabetes()
    X = ds.data.astype(float); y = ds.target.astype(float)
    feat = list(getattr(ds, "feature_names", [f"f{i}" for i in range(X.shape[1])]))
    pack = _split_pack_from_xy(X, y, "diabetes", "reg", seed)
    return DatasetPack(pack.name, pack.task, pack.Xtr, pack.ytr, pack.Xva, pack.yva, pack.Xte, pack.yte, feat)

def load_iris(seed=0):
    ds = skl_load_iris()
    X = ds.data.astype(float); y = ds.target.astype(int)
    feat = list(ds.feature_names)
    pack = _split_pack_from_xy(X, y, "iris", "clf", seed)
    return DatasetPack(pack.name, pack.task, pack.Xtr, pack.ytr, pack.Xva, pack.yva, pack.Xte, pack.yte, feat)

def load_wine(seed=0):
    ds = skl_load_wine()
    X = ds.data.astype(float); y = ds.target.astype(int)
    feat = list(ds.feature_names)
    pack = _split_pack_from_xy(X, y, "wine", "clf", seed)
    return DatasetPack(pack.name, pack.task, pack.Xtr, pack.ytr, pack.Xva, pack.yva, pack.Xte, pack.yte, feat)

def load_breast_cancer(seed=0):
    ds = skl_load_breast_cancer()
    X = ds.data.astype(float); y = ds.target.astype(int)
    feat = list(ds.feature_names)
    pack = _split_pack_from_xy(X, y, "breast_cancer", "clf", seed)
    return DatasetPack(pack.name, pack.task, pack.Xtr, pack.ytr, pack.Xva, pack.yva, pack.Xte, pack.yte, feat)

def load_credit_g(seed=0):
    try:
        ds = fetch_openml("credit-g", version=1, as_frame=True)
        dfX = ds.data.select_dtypes(include=[np.number]).copy()
        y = (ds.target == 'good').astype(int).to_numpy()
        X = dfX.to_numpy(float)
        return _split_pack_from_xy(X, y, "credit_g", "clf", seed)
    except Exception:
        return load_synthetic_clf(seed)

def load_covertype_small(seed=0):
    try:
        ds = fetch_openml("Covertype", version=3, as_frame=True)
        X = ds.data.select_dtypes(include=[np.number]).to_numpy(float)
        y = ds.target.astype('category').cat.codes.to_numpy()
        # downsample to keep runtime reasonable
        rs = check_random_state(seed)
        idx = rs.choice(np.arange(X.shape[0]), size=min(15000, X.shape[0]), replace=False)
        return _split_pack_from_xy(X[idx], y[idx], "covertype_small", "clf", seed)
    except Exception:
        return load_synthetic_clf(seed)

def load_phishing(seed=0):
    try:
        ds = fetch_openml("PhishingWebsites", version=1, as_frame=True)
        dfX = ds.data.select_dtypes(include=[np.number]).copy()
        X = dfX.to_numpy(float)
        y = (ds.target == 'phishing').astype(int).to_numpy()
        return _split_pack_from_xy(X, y, "phishing", "clf", seed)
    except Exception:
        return load_synthetic_clf(seed)

# --- Synthetic families + feature-like modalities ---
def load_synthetic_clf(seed=0):
    X, y = make_classification(n_samples=3000, n_features=40, n_informative=10,
                               n_redundant=10, n_repeated=5, n_classes=2, random_state=seed)
    return _split_pack_from_xy(X, y, "synthetic_clf", "clf", seed)

def load_synthetic_reg(seed=0):
    X, y = make_regression(n_samples=3000, n_features=40, n_informative=10, noise=1.5, random_state=seed)
    return _split_pack_from_xy(X, y, "synthetic_reg", "reg", seed)

def _dominant_freq(x, fs=1.0):
    f, Pxx = periodogram(x, fs=fs)
    return f[np.argmax(Pxx)] if len(Pxx)>0 and np.isfinite(Pxx).any() else 0.0

def _spec_entropy(x, fs=1.0):
    f, Pxx = periodogram(x, fs=fs); p = Pxx + 1e-12; p /= p.sum()
    return float(entropy(p))

def _ts_to_feats(T, fs=1.0):
    T = np.asarray(T, float)
    return np.array([T.mean(), T.std(), T.min(), T.max(),
                     _dominant_freq(T, fs), _spec_entropy(T, fs),
                     np.mean(np.diff(T)), np.std(np.diff(T))], float)

def _gen_ts_dataset(n_samples=2000, length=256, n_channels=1, seed=0, task="clf"):
    rs = check_random_state(seed); fs=1.0; X_feats=[]; y=[]
    for _ in range(n_samples):
        label = rs.randint(0,2) if task=="clf" else None
        chans=[]
        for _c in range(n_channels):
            freq = 0.05 + 0.2*rs.rand() + (0.1 if (task=="clf" and label==1) else 0.0)
            t = np.arange(length)/fs
            sig = (np.sin(2*np.pi*freq*t)
                   + 0.5*np.sin(2*np.pi*(freq*2)*t + rs.rand()*2*np.pi)
                   + 0.3*rs.randn(length))
            chans.append(sig)
        chans = np.stack(chans, axis=0)
        feats = np.concatenate([_ts_to_feats(ch, fs) for ch in chans], 0); X_feats.append(feats)
        y.append(label if task=="clf" else feats.sum()+rs.randn()*0.5)
    X = np.vstack(X_feats); y = np.array(y, dtype=int if task=="clf" else float)
    return X, y

def load_ecg_heartbeat(seed=0):
    X, y = _gen_ts_dataset(2500, 256, 1, seed, "clf")
    return _split_pack_from_xy(X, y, "ecg_heartbeat", "clf", seed)

def load_fashion_mnist_features(seed=0):
    rs = check_random_state(seed); n=5000; d=128; y=rs.randint(0,10,size=n)
    Z = rs.randn(n,16) + (y[:,None]*0.15); W = rs.randn(16,d)
    X = Z@W + 0.1*rs.randn(n,d)
    return _split_pack_from_xy(X, y, "fashion_mnist_features", "clf", seed)

def load_protein_structure(seed=0):
    X, y = make_regression(4000,120,20,noise=2.0,random_state=seed)
    return _split_pack_from_xy(X, y, "protein_structure", "reg", seed)

def load_stock_technical(seed=0):
    rs=check_random_state(seed); n=4000; T=60
    prices = 100 + np.cumsum(rs.randn(n,T),1); rets = np.diff(prices,1)
    vol=rets.std(1); mom=prices[:,-1]-prices[:,0]
    skew=((rets-rets.mean(1,keepdims=True))**3).mean(1); kurt=((rets-rets.mean(1,keepdims=True))**4).mean(1)
    feats=np.c_[rets.mean(1),vol,mom,skew,kurt]; y=(rs.randn(n)+0.2*np.sign(mom)>0).astype(int)
    return _split_pack_from_xy(feats,y,"stock_technical","clf",seed)

def load_sensor_fusion(seed=0):
    X, y = _gen_ts_dataset(3000,128,3,seed,"clf")
    return _split_pack_from_xy(X, y, "sensor_fusion", "clf", seed)

def load_gene_expression(seed=0):
    rs=check_random_state(seed); n=800; d=2000; y=rs.randint(0,2,size=n)
    base=rs.randn(n,d)*0.5; signal=np.zeros((n,d))
    idx=rs.choice(d,30,replace=False); signal[y==1][:,idx]=1.5
    X=base+signal+0.1*rs.randn(n,d)
    return _split_pack_from_xy(X,y,"gene_expression","clf",seed)

def load_network_topology(seed=0):
    rs=check_random_state(seed); n=3000
    deg=rs.gamma(2.0,2.0,n); cl=rs.beta(2,5,n); asst=rs.uniform(-0.5,0.5,n); tri=rs.poisson(5,n); btw=rs.exponential(1.0,n)
    X=np.c_[deg,cl,asst,tri,btw]; y=(0.3*deg+0.8*cl-0.5*asst+0.1*tri+0.2*btw+rs.randn(n)*0.5>1.8).astype(int)
    return _split_pack_from_xy(X,y,"network_topology","clf",seed)

def load_audio_mfcc(seed=0):
    rs=check_random_state(seed); n=3500; bands=20; y=rs.randint(0,5,size=n); base=rs.randn(n,bands)
    for c in range(5): base[y==c]+=(c-2)*0.25
    X=base+0.1*rs.randn(n,bands)
    return _split_pack_from_xy(X,y,"audio_mfcc","clf",seed)

def load_weather_station(seed=0):
    rs=check_random_state(seed); n=5000
    pressure=rs.normal(1013,8,n); humidity=rs.uniform(15,95,n); wind=rs.gamma(2.0,1.2,n); cloud=rs.uniform(0,1,n); doy=rs.randint(1,366,n)
    temp = 10 + 10*np.sin(2*np.pi*doy/365) - 0.01*(pressure-1013) - 0.05*(humidity-50) - 0.3*cloud + 0.2*wind + rs.randn(n)
    X=np.c_[pressure,humidity,wind,cloud,doy]; y=temp
    return _split_pack_from_xy(X,y,"weather_station","reg",seed)

def load_eeg_signals(seed=0):
    X, y = _gen_ts_dataset(3000,256,4,seed,"clf")
    return _split_pack_from_xy(X,y,"eeg_signals","clf",seed)

def load_satellite_ndvi(seed=0):
    rs=check_random_state(seed); n=4000
    bands=rs.uniform(0,1,(n,6)); sun=rs.uniform(0,70,n); view=rs.uniform(0,30,n)
    ndvi=(bands[:,3]-bands[:,2])/(bands[:,3]+bands[:,2]+1e-6); y=ndvi+0.001*(sun-35)-0.001*(view-15)+rs.randn(n)*0.02
    X=np.c_[bands,sun,view]
    return _split_pack_from_xy(X,y,"satellite_ndvi","reg",seed)

def load_industrial_process(seed=0):
    rs=check_random_state(seed); n=4500
    s1=rs.normal(0,1,n); s2=rs.normal(0,1.2,n); s3=rs.normal(0.5,0.8,n); sp=rs.uniform(-1,1,n); noise=rs.randn(n)*0.3
    y=2.0+1.5*s1-0.7*s2+0.9*s3+1.2*sp+noise; X=np.c_[s1,s2,s3,sp]
    return _split_pack_from_xy(X,y,"industrial_process","reg",seed)

def load_social_network(seed=0):
    rs=check_random_state(seed); n=5000
    posts=rs.poisson(3,n); friends=rs.poisson(120,n); likes=rs.poisson(50,n); comments=rs.poisson(12,n); shares=rs.poisson(5,n)
    X=np.c_[posts,friends,likes,comments,shares]
    y=(0.01*friends+0.03*likes+0.05*comments+0.08*shares+0.2*posts+rs.randn(n)*0.5>3.0).astype(int)
    return _split_pack_from_xy(X,y,"social_network","clf",seed)

def load_cyber_security(seed=0):
    X,y=make_classification(5000,30,8,10,weights=[0.8,0.2],class_sep=1.5,random_state=seed)
    return _split_pack_from_xy(X,y,"cyber_security","clf",seed)

def load_mixed_modal(seed=0):
    rs=check_random_state(seed)
    X_tab,y=make_classification(4000,20,6,6,random_state=seed); X_spec=[]
    for _ in range(X_tab.shape[0]):
        t=np.arange(64); sig=np.sin(2*np.pi*(0.08+0.1*rs.rand())*t)+0.3*rs.randn(64)
        X_spec.append(_ts_to_feats(sig))
    X=np.c_[X_tab,np.vstack(X_spec)]
    return _split_pack_from_xy(X,y,"mixed_modal","clf",seed)

# --- Registry: 29 datasets ---
ALL_DATASETS = {
    # OpenML/text + classics
    "20ng_bin":load_20ng_binary, "adult":load_adult, "har6":load_har6,
    "digits8x8":load_digits8x8, "california":load_california, "diabetes":load_diabetes,
    "iris":load_iris, "wine":load_wine, "breast_cancer":load_breast_cancer,
    "credit_g":load_credit_g, "covertype_small":load_covertype_small, "phishing":load_phishing,
    # synthetic & modality-like
    "synthetic_clf":load_synthetic_clf, "synthetic_reg":load_synthetic_reg,
    "ecg_heartbeat":load_ecg_heartbeat, "fashion_mnist_features":load_fashion_mnist_features,
    "protein_structure":load_protein_structure, "stock_technical":load_stock_technical,
    "sensor_fusion":load_sensor_fusion, "gene_expression":load_gene_expression,
    "network_topology":load_network_topology, "audio_mfcc":load_audio_mfcc,
    "weather_station":load_weather_station, "eeg_signals":load_eeg_signals,
    "satellite_ndvi":load_satellite_ndvi, "industrial_process":load_industrial_process,
    "social_network":load_social_network, "cyber_security":load_cyber_security,
    "mixed_modal":load_mixed_modal,
}
assert len(ALL_DATASETS) == 29, f"Expect 29 datasets, got {len(ALL_DATASETS)}"

# %%
# ============ 6) CSV & Plot Helpers ============

def ensure_out(outdir="out"):
    os.makedirs(outdir, exist_ok=True)

def append_rows(path: str, header: List[str], rows: List[List]):
    write_header = not os.path.exists(path)
    with open(path, "a", newline="") as f:
        wr = csv.writer(f)
        if write_header: wr.writerow(header)
        if rows: wr.writerows(rows)

# %%
# ============ 7) Per-dataset Pipeline ============

def run_on_dataset(pack: DatasetPack, ks=(3,5,8,12), seed: int = 0,
                   do_pfi: bool = True, outdir="out") -> Dict[str, np.ndarray]:
    ensure_out(outdir)
    Ntot = pack.Xtr.shape[0]+pack.Xva.shape[0]+pack.Xte.shape[0]
    print(f"[{pack.name}] task={pack.task}, N={Ntot}, D={pack.Xtr.shape[1]}")

    # (1) Train base model
    base = train_gboost(pack.task, pack.Xtr, pack.ytr, pack.Xva, pack.yva, seed=seed)
    if base is None:
        print(f"  -> skip: base model failed.")
        return {}

    # (2) Base metric
    yhat_test = base.predict(pack.Xte.astype(np.float32))
    base_metric = accuracy_score(pack.yte, yhat_test) if pack.task=="clf" else r2_score(pack.yte, yhat_test)
    append_rows(os.path.join(outdir, "base_model_metrics.csv"),
                ["dataset","task","test_metric_name","test_metric_value"],
                [[pack.name, pack.task, "accuracy" if pack.task=="clf" else "r2", base_metric]])

    # (3) Scores for all methods
    p_va = predict_scores(pack.task, base, pack.Xva)
    scores: Dict[str, np.ndarray] = {
        "ExCIR":        excir_feature_scores(pack.Xva, p_va),
        "TreeGain":     tree_gain_scores(base, pack.Xva.shape[1]),
        "PDP-var":      pdp_var_scores(base, pack.Xva),
        "MI(pred)":     mi_pred_scores(pack.Xva, p_va, task=pack.task),
        "MI(label)":    mi_label_scores(pack.Xva, pack.yva, task=pack.task),
        "Surrogate-LR": surrogate_lr_scores(pack.Xva, p_va),
    }
    if do_pfi:
        scores["PFI"] = pfi_scores(base, pack.Xva, pack.yva, n_repeats=5, seed=seed)

    # (4) Export feature importances
    feat_rows = []
    for mname, vec in scores.items():
        for j, val in enumerate(vec):
            fname = pack.feature_names[j] if j < len(pack.feature_names) else f"f{j}"
            feat_rows.append([pack.name, mname, j, fname, float(val)])
    append_rows(os.path.join(outdir, "feature_importances.csv"),
                ["dataset","method","feature_index","feature_name","score"],
                feat_rows)

    # (5) AOPC (insertion/deletion)
    curves_rows, summary_rows = [], []
    for mname, scr in scores.items():
        fr, ins, dele = aopc_curves(pack.task, base, pack.Xte, pack.yte, scr, steps=9)
        for f, a, d in zip(fr, ins, dele):
            curves_rows.append([pack.name, mname, float(f), float(a), float(d)])
        auc_ins = float(np.trapz(ins, fr))
        auc_del = float(np.trapz(dele, fr))
        combined = 0.5*(auc_ins + (1.0 - auc_del))
        summary_rows.append([pack.name, mname, auc_ins, auc_del, combined])
    append_rows(os.path.join(outdir, "aopc_curves.csv"),
                ["dataset","method","fraction_kept","insertion","deletion"], curves_rows)
    append_rows(os.path.join(outdir, "aopc_summary.csv"),
                ["dataset","method","auc_insertion","auc_deletion","combined"], summary_rows)

    # (6) Top-k sufficiency (retrain on top-k features by each method)
    topk_rows = []
    for m, v in scores.items():
        ranks = rank_from_scores(v)
        for k in ks:
            keep = ranks[:k]
            params = dict(n_estimators=100, random_state=seed, tree_method='gpu_hist',
                          gpu_id=0, validate_parameters=True, n_jobs=1)
            Xtr_k = np.vstack([pack.Xtr, pack.Xva])[:, keep].astype(np.float32)
            ytr_k = np.r_[pack.ytr, pack.yva]
            model_k = _get_xgb_model(pack.task, ytr_k, params)
            try:
                model_k.fit(Xtr_k, ytr_k)
            except xgb.core.XGBoostError:
                params['tree_method']='hist'; params.pop('gpu_id', None)
                model_k = _get_xgb_model(pack.task, ytr_k, params)
                try: model_k.fit(Xtr_k, ytr_k)
                except Exception: model_k = None
            except Exception:
                model_k = None

            metric_val = np.nan
            if model_k is not None:
                try:
                    yhat = model_k.predict(pack.Xte[:, keep].astype(np.float32))
                    metric_val = accuracy_score(pack.yte, yhat) if pack.task=="clf" else r2_score(pack.yte, yhat)
                except Exception:
                    metric_val = np.nan
            topk_rows.append([pack.name, m, k, "accuracy" if pack.task=="clf" else "r2", metric_val])
    append_rows(os.path.join(outdir, "summary_topk.csv"),
                ["dataset","method","k","metric_name","metric_value"], topk_rows)

    # (7) Stability under tiny i.i.d. noise
    cirs, j8 = stability_noise_eval(lambda X, p: excir_feature_scores(X, p), pack.Xva, p_va, repeats=15, sigma=0.01)
    stab_rows = [[pack.name, int(i), float(c), float(j)] for i, (c, j) in enumerate(zip(cirs, j8))]
    append_rows(os.path.join(outdir, "stability_noise.csv"),
                ["dataset","repeat_idx","cir_under_noise","jaccard_at_8_under_noise"], stab_rows)
    plt.figure(figsize=(6.6,4.0))
    plt.subplot(1,2,1); plt.hist(cirs, bins=10); plt.title('Spearman vs noisy'); plt.xlabel('Spearman'); plt.ylabel('count')
    plt.subplot(1,2,2); plt.hist(j8, bins=10); plt.title('Jaccard@8 vs noisy'); plt.xlabel('overlap')
    plt.tight_layout(); plt.savefig(os.path.join(outdir, f"stability_{pack.name}.png"), dpi=160); plt.close()

    # (8) Lightweight transfer checks (fractions)
    F = np.array([0.2,0.3,0.4,0.5,0.75,1.0])
    base_full = train_gboost(pack.task, pack.Xtr, pack.ytr, pack.Xva, pack.yva, seed=seed)
    if base_full is None:
        return scores
    p_full = predict_scores(pack.task, base_full, pack.Xva); s_full = excir_feature_scores(pack.Xva, p_full)
    Xbig = np.vstack([pack.Xtr, pack.Xva]); ybig = np.r_[pack.ytr, pack.yva]

    Times, CIRs, Resid, KLs, J8 = [], [], [], [], []
    rs = check_random_state(seed)
    for f in F:
        t0 = time.time()
        n_rows = int(f*Xbig.shape[0])
        idx = rs.choice(np.arange(Xbig.shape[0]), size=n_rows, replace=False)

        params = dict(n_estimators=30, random_state=seed, tree_method='gpu_hist',
                      gpu_id=0, validate_parameters=True, n_jobs=1)
        m = _get_xgb_model(pack.task, ybig, params)
        try:
            m.fit(Xbig[idx].astype(np.float32), ybig[idx])
        except xgb.core.XGBoostError:
            params['tree_method']='hist'; params.pop('gpu_id', None)
            m = _get_xgb_model(pack.task, ybig, params)
            try: m.fit(Xbig[idx].astype(np.float32), ybig[idx])
            except Exception: m = None
        except Exception:
            m = None

        if m is None:
            Times.append(time.time()-t0); CIRs += [np.nan]; Resid += [np.nan]; KLs += [np.nan]; J8 += [np.nan]
            continue

        p_sub = predict_scores(pack.task, m, pack.Xva)
        s_sub = excir_feature_scores(pack.Xva, p_sub)
        Times.append(time.time()-t0)
        CIRs.append(spearman_rank_corr(s_full, s_sub))
        Resid.append(projection_alignment_residual(p_full, p_sub))
        KLs.append(kde_kl_symmetric(p_full, p_sub))
        J8.append(jaccard_at_k(s_full.argsort()[::-1], s_sub.argsort()[::-1], k=min(8, s_full.shape[0])))

    lw_rows = [[pack.name, float(f), float(t), float(c if c==c else np.nan),
                float(r if r==r else np.nan), float(k if k==k else np.nan),
                float(j if j==j else np.nan)]
               for f, t, c, r, k, j in zip(F, Times, CIRs, Resid, KLs, J8)]
    append_rows(os.path.join(outdir, "lightweight_checks.csv"),
                ["dataset","fraction_rows","wall_time_sec","cir_agreement","proj_residual","sym_kl","jaccard_at_8"],
                lw_rows)

    # Pareto & runtime plots
    plt.figure(figsize=(6.5,5.0))
    scatter = plt.scatter(Times, CIRs, s=300*np.maximum(np.array(J8), 0.05), alpha=0.7, c=J8, cmap='viridis')
    for i, f in enumerate(F): plt.annotate(f"{int(100*f)}%", (Times[i], CIRs[i]), xytext=(5,5), textcoords='offset points', fontsize=9)
    plt.xlabel('Wall time (s)'); plt.ylabel('Spearman(full vs light)'); plt.title(f'Pareto (ExCIR) — {pack.name}')
    plt.colorbar(scatter, label='Jaccard@8'); plt.grid(alpha=0.3); plt.tight_layout()
    plt.savefig(os.path.join(outdir, f"pareto_{pack.name}.png"), dpi=160); plt.close()

    plt.figure(figsize=(6.2,4.2))
    plt.plot((100*F).astype(int), Times, marker='o'); plt.xlabel('Fraction rows kept (%)'); plt.ylabel('Wall time (s)')
    plt.title(f'Runtime vs fraction — {pack.name}'); plt.grid(alpha=0.3); plt.tight_layout()
    plt.savefig(os.path.join(outdir, f"runtime_{pack.name}.png"), dpi=160); plt.close()

    # (9) AOPC figure for ExCIR (visual)
    fr, ins, dele = aopc_curves(pack.task, base, pack.Xte, pack.yte, scores["ExCIR"], steps=9)
    plt.figure(figsize=(6.5,4.3))
    plt.plot(100*fr, ins, marker='o', label='Insertion (keep top-%)')
    plt.plot(100*fr, dele, marker='o', label='Deletion (zero top-%)')
    plt.xlabel('Revealed / removed top-% features'); plt.ylabel('Test ' + ('Accuracy' if pack.task=='clf' else 'R$^2$'))
    plt.title(f'AOPC (ExCIR) — {pack.name}'); plt.legend(); plt.tight_layout()
    plt.savefig(os.path.join(outdir, f"aopc_{pack.name}.png"), dpi=160); plt.close()

    # (10) digits: class-conditioned ExCIR example
    if pack.name == "digits8x8":
        clf = LogisticRegression(max_iter=2000, multi_class='multinomial', solver='lbfgs', random_state=seed)
        clf.fit(np.vstack([pack.Xtr, pack.Xva]), np.r_[pack.ytr, pack.yva]); Pva = clf.predict_proba(pack.Xva)
        s_c = excir_feature_scores(pack.Xva, Pva[:, 3])  # class 3
        H = s_c.reshape(8,8)
        append_rows(os.path.join(outdir, "digits_excir_class3_map.csv"),
                    [f"col{j}" for j in range(8)],
                    [list(map(float, row)) for row in H])
        plt.figure(figsize=(4.5,4.5)); plt.imshow(H, cmap='viridis'); plt.colorbar()
        plt.title('Class-conditioned ExCIR (digit 3)'); plt.tight_layout()
        plt.savefig(os.path.join(outdir, "digits_excir_class3.png"), dpi=160); plt.close()

    return scores

# %%
# ============ 8) Runner (All 29 datasets) ============

def run_all(datasets: str = "all", seed: int = 0, ks=(3,5,8,12), outdir="out", do_pfi=True):
    ensure_out(outdir)
    names = [s.strip() for s in datasets.split(",") if s.strip()]
    if not names or 'all' in [n.lower() for n in names]:
        names = list(ALL_DATASETS.keys())

    print(f"--- ExCIR Benchmark on {len(names)} datasets (seed={seed}) ---")
    print(f"Using XGBoost (GPU when available → CPU fallback).")
    print(f"Writing CSV/PNG to: {os.path.abspath(outdir)}")
    for name in names:
        if name not in ALL_DATASETS:
            print(f"[warn] dataset '{name}' not found, skipping.")
            continue
        try:
            pack = ALL_DATASETS[name](seed=seed)
            run_on_dataset(pack, ks=ks, seed=seed, do_pfi=do_pfi, outdir=outdir)
        except Exception as e:
            print(f"[ERROR] {name}: {e}")
            import traceback; traceback.print_exc(file=sys.stdout)

# %%
# ============ 9) Main ============

def main():
    run_all(datasets="all", seed=0, ks=(3,5,8,12), outdir="out", do_pfi=True)

if __name__ == "__main__":
    main()


In [6]:
# -*- coding: utf-8 -*-
# ExCIR — Reproducible cross-domain benchmark (29 datasets)
# - Deterministic setup
# - CIR / BlockCIR
# - Agreement metrics (Jaccard@k, Precision@k, Spearman)
# - AOPC helpers
# - Full plotting suite (centering ablation, BlockCIR ablation, Pareto mini-grid)
# - Datasets: 24 from your earlier script + 5 small structured synthetics to reach 29

# =====================
# 0) ENV & DETERMINISM
# =====================
import os, sys, csv, time, warnings, math, gc
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde, entropy, spearmanr
from scipy.signal import periodogram

# XGBoost (GPU fallback -> CPU hist)
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, r2_score
from sklearn.inspection import permutation_importance, partial_dependence
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.utils import check_random_state

# Scikit-learn datasets (renamed to avoid recursion)
from sklearn.datasets import (
    fetch_openml, fetch_california_housing, fetch_20newsgroups,
    load_digits, load_iris as skl_load_iris, load_wine as skl_load_wine,
    load_diabetes as skl_load_diabetes, load_breast_cancer as skl_load_breast_cancer,
    make_classification, make_regression
)
from sklearn.feature_extraction.text import TfidfVectorizer

# Threads (optional cap for reproducibility on BLAS/OpenMP)
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("VECLIB_MAXIMUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

GLOBAL_SEED = 0
rng_global = np.random.RandomState(GLOBAL_SEED)

def seed_everything(seed=0):
    np.random.seed(seed)
seed_everything(GLOBAL_SEED)

# I/O helpers
def ensure_out(outdir="out"): os.makedirs(outdir, exist_ok=True)

def append_rows(path: str, header: List[str], rows: List[List]):
    write_header = not os.path.exists(path)
    with open(path, "a", newline="") as f:
        wr = csv.writer(f)
        if write_header: wr.writerow(header)
        wr.writerows(rows)

# =========================
# 1) CORE: CIR/BlockCIR
# =========================
def _midmean(x: np.ndarray) -> float:
    """Mid-mean (trimmed mean on central 50%)."""
    x = np.asarray(x, float).ravel()
    q1, q3 = np.percentile(x, [25, 75])
    mid = x[(x >= q1) & (x <= q3)]
    return float(mid.mean() if mid.size else x.mean())

def _center_vec(a: np.ndarray, method: str = "midmean") -> np.ndarray:
    """Center a vector by midmean/median/mean."""
    if method == "midmean":
        return a - _midmean(a)
    if method == "median":
        return a - np.median(a)
    if method == "mean":
        return a - a.mean()
    raise ValueError("method must be 'midmean' | 'median' | 'mean'")

def excir_scores(
    X: np.ndarray,
    pred: np.ndarray,
    center: str = "midmean",
    eps: float = 1e-12
) -> np.ndarray:
    """
    ExCIR per-feature scores (bounded, correlation-aware).
    """
    X = np.asarray(X, float)
    p = np.asarray(pred, float).ravel()
    n, d = X.shape
    assert p.shape[0] == n

    # robust centering once
    y0 = _center_vec(p, center)
    # feature-wise centers: same scalar applied to the entire column (midmean of column)
    X0 = X.copy()
    if center == "midmean":
        # column-wise midmeans, streamed style
        mcols = np.array([_midmean(X[:, j]) for j in range(d)], float)
    elif center == "median":
        mcols = np.median(X, axis=0)
    elif center == "mean":
        mcols = X.mean(axis=0)
    X0 = X0 - mcols

    # one pass accumulators
    # p_ij = x_ij * y_i
    N = np.einsum("ij,i->j", X0, y0)  # sum x_ij*y_i
    D = np.einsum("ij,i->j", np.abs(X0), np.abs(y0)) + eps  # sum |x_ij*y_i|
    return 0.5 * (1.0 + (N / D))

def blockcir_scores(
    X: np.ndarray,
    pred: np.ndarray,
    blocks: List[List[int]],
    center: str = "midmean",
    eps: float = 1e-12
) -> np.ndarray:
    """
    BlockCIR: group-wise ExCIR with the same bounded ratio.
    """
    X = np.asarray(X, float)
    p = np.asarray(pred, float).ravel()
    n, d = X.shape
    y0 = _center_vec(p, center)

    if center == "midmean":
        mcols = np.array([_midmean(X[:, j]) for j in range(d)], float)
    elif center == "median":
        mcols = np.median(X, axis=0)
    else:
        mcols = X.mean(axis=0)
    X0 = X - mcols

    out = []
    for G in blocks:
        G = list(G)
        XG = X0[:, G]
        # N_G = sum_i sum_j in G  x_ij*y_i == (sum_j x_ij)*y_i summed over i
        # D_G = sum_i sum_j in G |x_ij*y_i|
        # vectorized:
        pG = XG * y0[:, None]
        NG = pG.sum()
        DG = np.abs(pG).sum() + eps
        out.append(0.5 * (1.0 + NG / DG))
    return np.array(out, float)

def auto_corr_blocks(
    X: np.ndarray,
    max_groups: int = 20,
    corr_thresh: float = 0.9
) -> List[List[int]]:
    """
    Build rough correlated blocks by agglomerating features whose absolute
    Pearson correlation > corr_thresh. Returns up to max_groups blocks.
    (Fast heuristic; works well enough for BlockCIR demo.)
    """
    X = np.asarray(X, float)
    d = X.shape[1]
    C = np.corrcoef(X, rowvar=False)
    used = np.zeros(d, bool)
    blocks = []
    for j in np.argsort(-np.nanmax(np.abs(C), axis=1)):
        if used[j]:
            continue
        group = [j]
        used[j] = True
        for k in range(d):
            if not used[k] and k != j and abs(C[j, k]) >= corr_thresh:
                used[k] = True
                group.append(k)
        blocks.append(group)
        if len(blocks) >= max_groups:
            break
    # Put any remaining singletons into tiny blocks if needed
    for j in range(d):
        if not used[j]:
            blocks.append([j])
            used[j] = True
        if len(blocks) >= max_groups:
            break
    return blocks

# ==========================
# 2) AGREEMENT & AOPC
# ==========================
def jaccard_topk(a_scores, b_scores, k=8):
    A = set(np.argsort(-a_scores)[:k]); B = set(np.argsort(-b_scores)[:k])
    return len(A & B) / (len(A | B) + 1e-12)

def precision_at_k(a_scores, b_scores, k=8):
    A = list(np.argsort(-a_scores)[:k]); B = set(np.argsort(-b_scores)[:k])
    return sum(1 for i in A if i in B) / float(max(k, 1))

def spearman_corr(a_scores, b_scores):
    # guard degenerate vectors
    if np.allclose(a_scores, a_scores[0]) and np.allclose(b_scores, b_scores[0]):
        return 1.0
    r, _ = spearmanr(a_scores, b_scores)
    return float(0.0 if np.isnan(r) else r)

def cir_pair(z: np.ndarray, s: np.ndarray, eps: float = 1e-12) -> float:
    """Pairwise CIR alignment in [0,1] for two score vectors (fast version)."""
    z = np.asarray(z, float).ravel(); s = np.asarray(s, float).ravel()
    mu_z, mu_s = z.mean(), s.mean()
    m = 0.5*(mu_z + mu_s)
    num = z.size * ((mu_z - m)**2 + (mu_s - m)**2)
    den = np.sum((z - m)**2) + np.sum((s - m)**2) + eps
    return float(num/den)

def projection_alignment_residual(y_full, y_light):
    y = np.asarray(y_full).ravel(); yp = np.asarray(y_light).ravel()
    n = y.shape[0]; Phi = np.c_[yp, np.ones(n)]
    theta, *_ = np.linalg.lstsq(Phi, y, rcond=None)
    yhat = Phi @ theta
    return float(np.linalg.norm(y - yhat) / (np.linalg.norm(y) + 1e-12))

def kde_kl_symmetric(y_full, y_light, grid_points=400):
    y = np.asarray(y_full).ravel(); yp = np.asarray(y_light).ravel()
    kde_p = gaussian_kde(y); kde_q = gaussian_kde(yp)
    both = np.r_[y, yp]; lo, hi = np.percentile(both, 0.5), np.percentile(both, 99.5)
    xs = np.linspace(lo, hi, grid_points)
    p = np.clip(kde_p(xs), 1e-12, None); q = np.clip(kde_q(xs), 1e-12, None)
    dx = (hi - lo)/max(grid_points-1, 1)
    kl_pq = float(np.sum(p*(np.log(p)-np.log(q))) * dx)
    kl_qp = float(np.sum(q*(np.log(q)-np.log(p))) * dx)
    return 0.5*(kl_pq + kl_qp)

def aopc_curves(task, model, Xte, yte, scores, steps=9):
    """Insertion↑ / Deletion↓ (0→100% top features)."""
    idx = np.argsort(-scores)
    n, d = Xte.shape; fracs = np.linspace(0, 1, steps)
    ins, dele = [], []
    for f in fracs:
        k = max(1, int(f*d)); keep = idx[:k]
        mask = np.zeros(d, bool); mask[keep] = True
        X_ins = np.where(mask, Xte, 0.0).astype(np.float32)
        X_del = np.where(mask, 0.0, Xte).astype(np.float32)
        y_ins = model.predict(X_ins); y_del = model.predict(X_del)
        if task == "clf":
            ins.append(accuracy_score(yte, y_ins)); dele.append(accuracy_score(yte, y_del))
        else:
            ins.append(r2_score(yte, y_ins)); dele.append(r2_score(yte, y_del))
    return fracs, np.asarray(ins), np.asarray(dele)

# ==========================
# 3) MODELS & SCORES
# ==========================
def _get_xgb_model(task, y_data, params):
    unique_classes = np.unique(y_data)
    if task == "clf":
        if len(unique_classes) > 2:
            return xgb.XGBClassifier(
                objective="multi:softmax", num_class=len(unique_classes),
                eval_metric="merror", **params
            )
        else:
            return xgb.XGBClassifier(
                objective="binary:logistic", eval_metric="logloss",
                use_label_encoder=False, **params
            )
    return xgb.XGBRegressor(objective="reg:squarederror", eval_metric="rmse", **params)

def train_xgb(task, Xtr, ytr, Xva, yva, seed=0):
    Xall = np.vstack([Xtr, Xva]).astype(np.float32)
    yall = np.r_[ytr, yva]
    params = dict(
        n_estimators=60, random_state=seed, n_jobs=1,
        tree_method="gpu_hist", gpu_id=0, validate_parameters=True
    )
    model = _get_xgb_model(task, yall, params)
    try:
        model.fit(Xall, yall)
        return model
    except xgb.core.XGBoostError:
        # CPU fallback
        params["tree_method"] = "hist"
        params.pop("gpu_id", None)
        model = _get_xgb_model(task, yall, params)
        model.fit(Xall, yall)
        return model

def pfi_scores(model, X, y, n_repeats=5, seed=0):
    try:
        return permutation_importance(
            model, X.astype(np.float32), y,
            n_repeats=n_repeats, random_state=seed
        ).importances_mean
    except Exception:
        return np.zeros(X.shape[1])

def tree_gain_scores(model, d):
    w = getattr(model, "feature_importances_", None)
    return w if (w is not None and len(w) == d) else np.zeros(d)

def pdp_var_scores(model, X, grid_points=15, random_state=0):
    d = X.shape[1]; scores = np.zeros(d)
    rs = check_random_state(random_state)
    idx = rs.choice(np.arange(X.shape[0]), size=min(1000, X.shape[0]), replace=False)
    Xs = X[idx]
    for j in range(d):
        try:
            pd = partial_dependence(
                model, Xs.astype(np.float32), features=[j],
                kind="average", grid_resolution=grid_points
            )
            scores[j] = np.var(pd.average[0])
        except Exception:
            scores[j] = 0.0
    return scores

def surrogate_lr_scores(X, teacher_scores, alpha=1.0):
    X = np.asarray(X, float); y = np.asarray(teacher_scores, float).ravel()
    reg = Ridge(alpha=alpha, random_state=0).fit(X, y)
    w = np.abs(reg.coef_)
    if w.ndim > 1: w = np.linalg.norm(w, axis=0)
    s = w / (w.sum() + 1e-12)
    return s

def mi_pred_scores(X, preds, task):
    X = np.asarray(X, float); p = np.asarray(preds, float).ravel()
    if task == "clf":
        z = (p > np.median(p)).astype(int)
        return mutual_info_classif(X, z, random_state=0)
    z = np.digitize(p, np.quantile(p, [1/3, 2/3]))
    return mutual_info_classif(X, z, random_state=0)

def mi_label_scores(X, y, task):
    return mutual_info_classif(X, y, random_state=0) if task=="clf" else mutual_info_regression(X, y, random_state=0)

# ==========================
# 4) DATASETS (29 total)
# ==========================
@dataclass
class DatasetPack:
    name: str
    task: str  # "clf" | "reg"
    Xtr: np.ndarray; ytr: np.ndarray
    Xva: np.ndarray; yva: np.ndarray
    Xte: np.ndarray; yte: np.ndarray
    feature_names: List[str]

def _standardize_fit(Xtr, Xva, Xte):
    ss = StandardScaler(with_mean=True, with_std=True)
    return ss.fit_transform(Xtr), ss.transform(Xva), ss.transform(Xte)

def _split_xy(X, y, task, seed=0):
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2,
                                          stratify=y if task=="clf" else None,
                                          random_state=seed)
    Xtr, Xva, ytr, yva = train_test_split(Xtr, ytr, test_size=0.25,
                                          stratify=ytr if task=="clf" else None,
                                          random_state=seed)
    Xtr_s, Xva_s, Xte_s = _standardize_fit(Xtr, Xva, Xte)
    feat_names = [f"f{i}" for i in range(X.shape[1])]
    return Xtr_s, ytr, Xva_s, yva, Xte_s, yte, feat_names

# -- Original 24 you used before --
def load_adult(seed=0):
    try:
        ds = fetch_openml("adult", version=2, as_frame=True)
        dfX = ds.data.select_dtypes(include=[np.number]).copy()
        y = (ds.target == '>50K').astype(int).to_numpy()
        X = dfX.to_numpy(float)
        Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed)
        Xtr, Xva, ytr, yva = train_test_split(Xtr, ytr, test_size=0.2, stratify=ytr, random_state=seed)
        Xtr, Xva, Xte = _standardize_fit(Xtr, Xva, Xte)
        return DatasetPack("adult","clf",Xtr,ytr,Xva,yva,Xte,yte,list(dfX.columns))
    except Exception:
        return load_synthetic_clf(seed)

def load_20ng_binary(seed=0):
    try:
        cats = ['comp.graphics','sci.space']
        tr = fetch_20newsgroups(subset='train', categories=cats, remove=('headers','footers','quotes'))
        te = fetch_20newsgroups(subset='test',  categories=cats, remove=('headers','footers','quotes'))
        tf = TfidfVectorizer(max_features=3000, ngram_range=(1,2), stop_words='english')
        Xtr_full = tf.fit_transform(tr.data).astype(np.float32).toarray()
        Xte = tf.transform(te.data).astype(np.float32).toarray()
        ytr_full, yte = tr.target, te.target
        Xtr, Xva, ytr, yva = train_test_split(Xtr_full, ytr_full, test_size=0.2, stratify=ytr_full, random_state=seed)
        return DatasetPack("20ng_bin","clf",Xtr,ytr,Xva,yva,Xte,yte,[f"tfidf_{i}" for i in range(Xtr.shape[1])])
    except Exception:
        return load_synthetic_clf(seed)

def load_har(seed=0):
    try:
        ds = fetch_openml("HAR", version=1, as_frame=True)
        X = ds.data.to_numpy(float); y = ds.target.astype('category').cat.codes.to_numpy()
        Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed)
        Xtr, Xva, ytr, yva = train_test_split(Xtr, ytr, test_size=0.2, stratify=ytr, random_state=seed)
        Xtr, Xva, Xte = _standardize_fit(Xtr, Xva, Xte)
        return DatasetPack("har6","clf",Xtr,ytr,Xva,yva,Xte,yte,[f"sensor_{i}" for i in range(X.shape[1])])
    except Exception:
        return load_synthetic_clf(seed)

def load_digits8x8(seed=0):
    dg = load_digits(); X = dg.data.astype(float)/16.0; y = dg.target
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed)
    Xtr, Xva, ytr, yva = train_test_split(Xtr, ytr, test_size=0.2, stratify=ytr, random_state=seed)
    Xtr, Xva, Xte = _standardize_fit(Xtr, Xva, Xte)
    return DatasetPack("digits8x8","clf",Xtr,ytr,Xva,yva,Xte,yte,[f"pix{i}" for i in range(X.shape[1])])

def load_california(seed=0):
    try:
        ds = fetch_california_housing()
        X = ds.data.astype(float); y = ds.target.astype(float)
        Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=seed)
        Xtr, Xva, ytr, yva = train_test_split(Xtr, ytr, test_size=0.2, random_state=seed)
        Xtr, Xva, Xte = _standardize_fit(Xtr, Xva, Xte)
        return DatasetPack("california","reg",Xtr,ytr,Xva,yva,Xte,yte,list(ds.feature_names))
    except Exception:
        return load_synthetic_reg(seed)

def load_diabetes(seed=0):
    ds = skl_load_diabetes(); X = ds.data.astype(float); y = ds.target.astype(float)
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=seed)
    Xtr, Xva, ytr, yva = train_test_split(Xtr, ytr, test_size=0.2, random_state=seed)
    Xtr, Xva, Xte = _standardize_fit(Xtr, Xva, Xte)
    feat = list(getattr(ds, "feature_names", [f"f{i}" for i in range(X.shape[1])]))
    return DatasetPack("diabetes","reg",Xtr,ytr,Xva,yva,Xte,yte,feat)

def load_iris(seed=0):
    ds = skl_load_iris(); X = ds.data.astype(float); y = ds.target.astype(int)
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed)
    Xtr, Xva, ytr, yva = train_test_split(Xtr, ytr, test_size=0.2, stratify=ytr, random_state=seed)
    Xtr, Xva, Xte = _standardize_fit(Xtr, Xva, Xte)
    return DatasetPack("iris","clf",Xtr,ytr,Xva,yva,Xte,yte,list(ds.feature_names))

def load_wine(seed=0):
    ds = skl_load_wine(); X = ds.data.astype(float); y = ds.target.astype(int)
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed)
    Xtr2, Xva2, ytr2, yva2 = train_test_split(Xtr, ytr, test_size=0.2, stratify=ytr, random_state=seed)
    Xtr_s, Xva_s, Xte_s = _standardize_fit(Xtr2, Xva2, Xte)
    return DatasetPack("wine","clf",Xtr_s,ytr2,Xva_s,yva2,Xte_s,yte,list(ds.feature_names))

def load_synthetic_clf(seed=0):
    X, y = make_classification(n_samples=3000, n_features=40, n_informative=10,
                               n_redundant=10, n_repeated=5, n_classes=2, random_state=seed)
    Xtr, ytr, Xva, yva, Xte, yte, feat = _split_xy(X, y, "clf", seed)
    return DatasetPack("synthetic_clf","clf",Xtr,ytr,Xva,yva,Xte,yte,[f"synth_clf_{i}" for i in range(X.shape[1])])

def load_synthetic_reg(seed=0):
    X, y = make_regression(n_samples=3000, n_features=40, n_informative=10, random_state=seed)
    Xtr, ytr, Xva, yva, Xte, yte, feat = _split_xy(X, y, "reg", seed)
    return DatasetPack("synthetic_reg","reg",Xtr,ytr,Xva,yva,Xte,yte,[f"synth_reg_{i}" for i in range(X.shape[1])])

# extra synthetic modalities you used before (matching your previous list)
def _dominant_freq(x, fs=1.0):
    f, Pxx = periodogram(x, fs=fs)
    return f[np.argmax(Pxx)] if len(Pxx)>0 and np.isfinite(Pxx).any() else 0.0
def _spec_entropy(x, fs=1.0):
    f, Pxx = periodogram(x, fs=fs); p = Pxx + 1e-12; p /= p.sum(); return float(entropy(p))
def _ts_to_feats(T, fs=1.0):
    T = np.asarray(T, float)
    return np.array([T.mean(), T.std(), T.min(), T.max(),
                     _dominant_freq(T, fs), _spec_entropy(T, fs),
                     np.mean(np.diff(T)), np.std(np.diff(T))], float)
def _gen_ts_dataset(n_samples=2000, length=256, n_channels=1, seed=0, task="clf"):
    rs = check_random_state(seed); fs=1.0; X_feats=[]; y=[]
    for _ in range(n_samples):
        label = rs.randint(0,2) if task=="clf" else None
        chans=[]
        for _c in range(n_channels):
            freq = 0.05 + 0.2*rs.rand() + (0.1 if (task=="clf" and label==1) else 0.0)
            t = np.arange(length)/fs
            sig = np.sin(2*np.pi*freq*t)+0.5*np.sin(2*np.pi*(freq*2)*t + rs.rand()*2*np.pi) + 0.3*rs.randn(length)
            chans.append(sig)
        chans = np.stack(chans, axis=0)
        feats = np.concatenate([_ts_to_feats(ch, fs) for ch in chans], 0); X_feats.append(feats)
        y.append(label if task=="clf" else feats.sum()+rs.randn()*0.5)
    X = np.vstack(X_feats); y = np.array(y, dtype=int if task=="clf" else float)
    return X, y
def _split_pack_from_xy(X, y, name, task, seed):
    Xtr, ytr, Xva, yva, Xte, yte, feat = _split_xy(X, y, task, seed)
    return DatasetPack(name, task, Xtr, ytr, Xva, yva, Xte, yte, [f"{name}_f{i}" for i in range(X.shape[1])])

def load_ecg_heartbeat(seed=0): X,y=_gen_ts_dataset(2500,256,1,seed,"clf"); return _split_pack_from_xy(X,y,"ecg_heartbeat","clf",seed)
def load_fashion_mnist_features(seed=0):
    rs = check_random_state(seed); n=5000; d=128; y=rs.randint(0,10,size=n)
    Z = rs.randn(n,16) + (y[:,None]*0.15); W = rs.randn(16,d); X = Z@W + 0.1*rs.randn(n,d)
    return _split_pack_from_xy(X,y,"fashion_mnist_features","clf",seed)
def load_protein_structure(seed=0): X,y=make_regression(4000,120,20,noise=2.0,random_state=seed); return _split_pack_from_xy(X,y,"protein_structure","reg",seed)
def load_stock_technical(seed=0):
    rs=check_random_state(seed); n=4000; T=60
    prices = 100 + np.cumsum(rs.randn(n,T),1); rets = np.diff(prices,1)
    vol=rets.std(1); mom=prices[:,-1]-prices[:,0]
    skew=((rets-rets.mean(1,keepdims=True))**3).mean(1); kurt=((rets-rets.mean(1,keepdims=True))**4).mean(1)
    feats=np.c_[rets.mean(1),vol,mom,skew,kurt]; y=(rs.randn(n)+0.2*np.sign(mom)>0).astype(int)
    return _split_pack_from_xy(feats,y,"stock_technical","clf",seed)
def load_sensor_fusion(seed=0): X,y=_gen_ts_dataset(3000,128,3,seed,"clf"); return _split_pack_from_xy(X,y,"sensor_fusion","clf",seed)
def load_gene_expression(seed=0):
    rs=check_random_state(seed); n=800; d=2000; y=rs.randint(0,2,size=n)
    base=rs.randn(n,d)*0.5; signal=np.zeros((n,d)); idx=rs.choice(d,30,replace=False); signal[y==1][:,idx]=1.5
    X=base+signal+0.1*rs.randn(n,d); return _split_pack_from_xy(X,y,"gene_expression","clf",seed)
def load_network_topology(seed=0):
    rs=check_random_state(seed); n=3000
    deg=rs.gamma(2.0,2.0,n); cl=rs.beta(2,5,n); asst=rs.uniform(-0.5,0.5,n); tri=rs.poisson(5,n); btw=rs.exponential(1.0,n)
    X=np.c_[deg,cl,asst,tri,btw]; y=(0.3*deg+0.8*cl-0.5*asst+0.1*tri+0.2*btw+rs.randn(n)*0.5>1.8).astype(int)
    return _split_pack_from_xy(X,y,"network_topology","clf",seed)
def load_audio_mfcc(seed=0):
    rs=check_random_state(seed); n=3500; bands=20; y=rs.randint(0,5,size=n); base=rs.randn(n,bands)
    for c in range(5): base[y==c]+=(c-2)*0.25
    X=base+0.1*rs.randn(n,bands); return _split_pack_from_xy(X,y,"audio_mfcc","clf",seed)
def load_weather_station(seed=0):
    rs=check_random_state(seed); n=5000
    pressure=rs.normal(1013,8,n); humidity=rs.uniform(15,95,n); wind=rs.gamma(2.0,1.2,n); cloud=rs.uniform(0,1,n); doy=rs.randint(1,366,n)
    temp = 10 + 10*np.sin(2*np.pi*doy/365) - 0.01*(pressure-1013) - 0.05*(humidity-50) - 0.3*cloud + 0.2*wind + rs.randn(n)
    X=np.c_[pressure,humidity,wind,cloud,doy]; y=temp; return _split_pack_from_xy(X,y,"weather_station","reg",seed)
def load_eeg_signals(seed=0): X,y=_gen_ts_dataset(3000,256,4,seed,"clf"); return _split_pack_from_xy(X,y,"eeg_signals","clf",seed)
def load_satellite_ndvi(seed=0):
    rs=check_random_state(seed); n=4000
    bands=rs.uniform(0,1,(n,6)); sun=rs.uniform(0,70,n); view=rs.uniform(0,30,n)
    ndvi=(bands[:,3]-bands[:,2])/(bands[:,3]+bands[:,2]+1e-6); y=ndvi+0.001*(sun-35)-0.001*(view-15)+rs.randn(n)*0.02
    X=np.c_[bands,sun,view]; return _split_pack_from_xy(X,y,"satellite_ndvi","reg",seed)
def load_industrial_process(seed=0):
    rs=check_random_state(seed); n=4500
    s1=rs.normal(0,1,n); s2=rs.normal(0,1.2,n); s3=rs.normal(0.5,0.8,n); sp=rs.uniform(-1,1,n); noise=rs.randn(n)*0.3
    y=2.0+1.5*s1-0.7*s2+0.9*s3+1.2*sp+noise; X=np.c_[s1,s2,s3,sp]
    return _split_pack_from_xy(X,y,"industrial_process","reg",seed)
def load_social_network(seed=0):
    rs=check_random_state(seed); n=5000
    posts=rs.poisson(3,n); friends=rs.poisson(120,n); likes=rs.poisson(50,n); comments=rs.poisson(12,n); shares=rs.poisson(5,n)
    X=np.c_[posts,friends,likes,comments,shares]
    y=(0.01*friends+0.03*likes+0.05*comments+0.08*shares+0.2*posts+rs.randn(n)*0.5>3.0).astype(int)
    return _split_pack_from_xy(X,y,"social_network","clf",seed)
def load_cyber_security(seed=0):
    X,y=make_classification(5000,30,8,10,weights=[0.8,0.2],class_sep=1.5,random_state=seed)
    return _split_pack_from_xy(X,y,"cyber_security","clf",seed)
def load_mixed_modal(seed=0):
    rs=check_random_state(seed)
    X_tab,y=make_classification(4000,20,6,6,random_state=seed); X_spec=[]
    for _ in range(X_tab.shape[0]):
        t=np.arange(64); sig=np.sin(2*np.pi*(0.08+0.1*rs.rand())*t)+0.3*rs.randn(64)
        X_spec.append(_ts_to_feats(sig))
    X=np.c_[X_tab,np.vstack(X_spec)]; return _split_pack_from_xy(X,y,"mixed_modal","clf",seed)

# + 5 small structured add-ons to reach 29
def load_breast_cancer(seed=0):
    ds = skl_load_breast_cancer(); X = ds.data.astype(float); y = ds.target.astype(int)
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed)
    Xtr, Xva, ytr, yva = train_test_split(Xtr, ytr, test_size=0.2, stratify=ytr, random_state=seed)
    Xtr, Xva, Xte = _standardize_fit(Xtr, Xva, Xte)
    return DatasetPack("breast_cancer","clf",Xtr,ytr,Xva,yva,Xte,yte,list(ds.feature_names))

def load_retail_demand(seed=0):
    rs=check_random_state(seed); n=5000
    price=rs.lognormal(mean=2.5, sigma=0.5, size=n)
    promo=rs.binomial(1, 0.3, size=n)
    season=rs.randint(1,13,size=n)
    trend=np.linspace(0,1,n)
    noise=rs.randn(n)*0.2
    demand = 3.0 - 0.5*np.log(price) + 0.8*promo + 0.2*np.sin(2*np.pi*season/12) + 0.5*trend + noise
    X=np.c_[price,promo,season,trend]; y=demand
    return _split_pack_from_xy(X,y,"retail_demand","reg",seed)

def load_traffic_flow(seed=0):
    rs=check_random_state(seed); n=4500
    hour=rs.randint(0,24,n); dow=rs.randint(0,7,n); rain=rs.binomial(1,0.2,n)
    base= 20 + 15*np.sin(2*np.pi*hour/24) + 5*(dow>=5) - 3*rain
    noise=rs.randn(n)*2
    y = base + noise
    X = np.c_[hour,dow,rain]
    return _split_pack_from_xy(X,y,"traffic_flow","reg",seed)

def load_energy_consumption(seed=0):
    rs=check_random_state(seed); n=4000
    temp=rs.normal(18,7,n); humidity=rs.uniform(20,90,n); hour=rs.randint(0,24,n); occupancy=rs.poisson(5,n)
    y = 0.4*occupancy + 0.2*hour - 0.1*(temp-20) + 0.05*(humidity-50) + rs.randn(n)
    X=np.c_[temp,humidity,hour,occupancy]
    return _split_pack_from_xy(X,y,"energy_consumption","reg",seed)

def load_air_quality(seed=0):
    rs=check_random_state(seed); n=4200
    pm10=rs.lognormal(2.0,0.5,n); no2=rs.lognormal(1.5,0.3,n); o3=rs.lognormal(1.3,0.3,n)
    wind=rs.gamma(2.0,1.0,n); humidity=rs.uniform(20,90,n)
    y = 0.5*pm10 + 0.2*no2 - 0.1*o3 - 0.05*wind + 0.02*humidity + rs.randn(n)
    X = np.c_[pm10,no2,o3,wind,humidity]
    return _split_pack_from_xy(X,y,"air_quality","reg",seed)

ALL_DATASETS = {
    # your original set (~24)
    "adult":load_adult,"20ng_bin":load_20ng_binary,"har6":load_har,"digits8x8":load_digits8x8,
    "california":load_california,"diabetes":load_diabetes,"iris":load_iris,"wine":load_wine,
    "synthetic_clf":load_synthetic_clf,"synthetic_reg":load_synthetic_reg,
    "ecg_heartbeat":load_ecg_heartbeat,"fashion_mnist_features":load_fashion_mnist_features,
    "protein_structure":load_protein_structure,"stock_technical":load_stock_technical,
    "sensor_fusion":load_sensor_fusion,"gene_expression":load_gene_expression,
    "network_topology":load_network_topology,"audio_mfcc":load_audio_mfcc,
    "weather_station":load_weather_station,"eeg_signals":load_eeg_signals,
    "satellite_ndvi":load_satellite_ndvi,"industrial_process":load_industrial_process,
    "social_network":load_social_network,"cyber_security":load_cyber_security,"mixed_modal":load_mixed_modal,
    # +5 to reach 29
    "breast_cancer":load_breast_cancer,"retail_demand":load_retail_demand,
    "traffic_flow":load_traffic_flow,"energy_consumption":load_energy_consumption
}
assert len(ALL_DATASETS) == 29, f"{len(ALL_DATASETS)} datasets detected — expected 29."

# ==========================
# 5) RUNNER
# ==========================
def run_dataset(
    pack: DatasetPack,
    ks=(3,5,8,12),
    center: str = "midmean",
    outdir="out",
    do_pfi: bool = True,
    do_blockcir: bool = True
) -> Dict[str, np.ndarray]:
    ensure_out(outdir)
    print(f"[{pack.name}] N={pack.Xtr.shape[0]+pack.Xva.shape[0]+pack.Xte.shape[0]}  D={pack.Xtr.shape[1]}  task={pack.task}")

    # Train base model
    base = train_xgb(pack.task, pack.Xtr, pack.ytr, pack.Xva, pack.yva, seed=GLOBAL_SEED)
    yhat_te = base.predict(pack.Xte.astype(np.float32))
    metric = accuracy_score(pack.yte, yhat_te) if pack.task=="clf" else r2_score(pack.yte, yhat_te)
    append_rows(os.path.join(outdir, "base_model_metrics.csv"),
                ["dataset","task","test_metric_name","test_metric_value"],
                [[pack.name, pack.task, "accuracy" if pack.task=="clf" else "r2", metric]])

    # Predictions on val for explainability
    p_va = (base.predict_proba(pack.Xva.astype(np.float32))[:,1] if (pack.task=="clf" and hasattr(base,"predict_proba"))
            else base.predict(pack.Xva.astype(np.float32)))
    # Scores
    scores: Dict[str, np.ndarray] = {
        "ExCIR":        excir_scores(pack.Xva, p_va, center=center),
        "TreeGain":     tree_gain_scores(base, pack.Xva.shape[1]),
        "PDP-var":      pdp_var_scores(base, pack.Xva),
        "MI(pred)":     mi_pred_scores(pack.Xva, p_va, task=pack.task),
        "MI(label)":    mi_label_scores(pack.Xva, pack.yva, task=pack.task),
        "Surrogate-LR": surrogate_lr_scores(pack.Xva, p_va),
    }
    if do_pfi:
        scores["PFI"] = pfi_scores(base, pack.Xva, pack.yva, n_repeats=5, seed=GLOBAL_SEED)

    # CSV: feature_importances
    feat_rows = []
    for mname, vec in scores.items():
        for j, val in enumerate(vec):
            fname = pack.feature_names[j] if j < len(pack.feature_names) else f"f{j}"
            feat_rows.append([pack.name, mname, j, fname, float(val)])
    append_rows(os.path.join(outdir, "feature_importances.csv"),
                ["dataset","method","feature_index","feature_name","score"],
                feat_rows)

    # AOPC per method (curves + summary)
    curves_rows, summary_rows = [], []
    for mname, scr in scores.items():
        fr, ins, dele = aopc_curves(pack.task, base, pack.Xte, pack.yte, scr, steps=9)
        for f, a, d in zip(fr, ins, dele):
            curves_rows.append([pack.name, mname, float(f), float(a), float(d)])
        auc_ins = float(np.trapz(ins, fr))
        auc_del = float(np.trapz(dele, fr))
        combined = 0.5*(auc_ins + (1.0 - auc_del))
        summary_rows.append([pack.name, mname, auc_ins, auc_del, combined])
    append_rows(os.path.join(outdir, "aopc_curves.csv"),
                ["dataset","method","fraction_kept","insertion","deletion"],
                curves_rows)
    append_rows(os.path.join(outdir, "aopc_summary.csv"),
                ["dataset","method","auc_insertion","auc_deletion","combined"],
                summary_rows)

    # Top-k sufficiency via model@k
    topk_rows = []
    for mname, v in scores.items():
        ranks = np.argsort(-v)
        for k in ks:
            keep = ranks[:k]
            Xtr_k = np.vstack([pack.Xtr, pack.Xva])[:, keep]
            ytr_k = np.r_[pack.ytr, pack.yva]

            params = dict(n_estimators=80, random_state=GLOBAL_SEED, n_jobs=1,
                          tree_method="gpu_hist", gpu_id=0, validate_parameters=True)
            model_k = _get_xgb_model(pack.task, ytr_k, params)
            try:
                model_k.fit(Xtr_k.astype(np.float32), ytr_k)
            except xgb.core.XGBoostError:
                params["tree_method"]="hist"; params.pop("gpu_id", None)
                model_k = _get_xgb_model(pack.task, ytr_k, params)
                model_k.fit(Xtr_k.astype(np.float32), ytr_k)

            yhat = model_k.predict(pack.Xte[:, keep].astype(np.float32))
            val = accuracy_score(pack.yte, yhat) if pack.task=="clf" else r2_score(pack.yte, yhat)
            topk_rows.append([pack.name, mname, k, "accuracy" if pack.task=="clf" else "r2", float(val)])
    append_rows(os.path.join(outdir, "summary_topk.csv"),
                ["dataset","method","k","metric_name","metric_value"],
                topk_rows)

    # Stability under small i.i.d. noise for ExCIR
    repeats, sigma = 15, 0.01
    base_scores = excir_scores(pack.Xva, p_va, center=center)
    cirs, j8s, p8s, srs = [], [], [], []
    for _ in range(repeats):
        noise = np.random.normal(0, sigma, size=pack.Xva.shape)
        s_noisy = excir_scores(pack.Xva + noise, p_va, center=center)
        cirs.append(cir_pair(base_scores, s_noisy))
        j8s.append(jaccard_topk(base_scores, s_noisy, k=min(8, base_scores.shape[0])))
        p8s.append(precision_at_k(base_scores, s_noisy, k=min(8, base_scores.shape[0])))
        srs.append(spearman_corr(base_scores, s_noisy))
    stab_rows = [[pack.name, int(i), float(c), float(j), float(p), float(sr)]
                 for i, (c, j, p, sr) in enumerate(zip(cirs, j8s, p8s, srs))]
    append_rows(os.path.join(outdir, "stability_noise.csv"),
                ["dataset","repeat_idx","cir","jaccard@8","precision@8","spearman"],
                stab_rows)

    # Lightweight-pareto (agreement vs cost)
    fractions = (0.2, 0.3, 0.4, 0.6, 0.8, 1.0)
    Xbig = np.vstack([pack.Xtr, pack.Xva]); ybig = np.r_[pack.ytr, pack.yva]
    F, Times, CIRs, Resid, KLs, J8 = [], [], [], [], [], []
    for f in fractions:
        t0 = time.time()
        rs = check_random_state(GLOBAL_SEED)
        idx = rs.choice(np.arange(Xbig.shape[0]), size=max(50, int(f*Xbig.shape[0])), replace=False)
        params = dict(n_estimators=40, random_state=GLOBAL_SEED, n_jobs=1,
                      tree_method="gpu_hist", gpu_id=0, validate_parameters=True)
        m = _get_xgb_model(pack.task, ybig, params)
        try:
            m.fit(Xbig[idx].astype(np.float32), ybig[idx])
        except xgb.core.XGBoostError:
            params["tree_method"]="hist"; params.pop("gpu_id", None)
            m = _get_xgb_model(pack.task, ybig, params)
            m.fit(Xbig[idx].astype(np.float32), ybig[idx])

        p_full = p_va
        p_sub = (m.predict_proba(pack.Xva.astype(np.float32))[:,1] if (pack.task=="clf" and hasattr(m,"predict_proba"))
                 else m.predict(pack.Xva.astype(np.float32)))
        s_full = base_scores
        s_sub  = excir_scores(pack.Xva, p_sub, center=center)

        Times.append(time.time()-t0); F.append(f)
        CIRs.append(cir_pair(s_full, s_sub))
        Resid.append(projection_alignment_residual(p_full, p_sub))
        KLs.append(kde_kl_symmetric(p_full, p_sub))
        J8.append(jaccard_topk(s_full, s_sub, k=min(8, s_full.shape[0])))

    lw_rows = [[pack.name, float(f), float(t), float(c), float(r), float(k), float(j)]
               for f,t,c,r,k,j in zip(F,Times,CIRs,Resid,KLs,J8)]
    append_rows(os.path.join(outdir, "lightweight_checks.csv"),
                ["dataset","fraction_rows","wall_time_sec","cir_agreement",
                 "proj_residual","sym_kl","jaccard_at_8"],
                lw_rows)

    # Figures: per-dataset (optional, keep light)
    # (A) AOPC ExCIR
    fr, ins, dele = aopc_curves(pack.task, base, pack.Xte, pack.yte, base_scores, steps=9)
    plt.figure(figsize=(6.2,4.2))
    plt.plot(100*fr, ins, marker='o', label='Insertion')
    plt.plot(100*fr, dele, marker='o', label='Deletion')
    plt.xlabel('Top-% features'); plt.ylabel('Accuracy' if pack.task=='clf' else 'R$^2$')
    plt.title(f'AOPC (ExCIR) — {pack.name}'); plt.legend(); plt.tight_layout()
    plt.savefig(os.path.join(outdir, f"aopc_{pack.name}.png"), dpi=160); plt.close()

    # (B) Agreement–cost (Pareto)
    plt.figure(figsize=(6.0,4.6))
    sc = plt.scatter(Times, CIRs, s=250*np.maximum(J8,0.05), c=J8, cmap='viridis', alpha=0.8)
    for i, f in enumerate(F): plt.annotate(f"{int(100*f)}%", (Times[i], CIRs[i]), xytext=(4,4), textcoords='offset points', fontsize=9)
    plt.xlabel("Wall time (s)"); plt.ylabel("CIR(full, light)")
    plt.title(f"Agreement–cost (ExCIR) — {pack.name}")
    plt.colorbar(sc, label='Jaccard@8'); plt.grid(alpha=0.3); plt.tight_layout()
    plt.savefig(os.path.join(outdir, f"pareto_{pack.name}.png"), dpi=160); plt.close()

    return scores

# ==========================
# 6) PAPER PLOTS (global)
# ==========================
def plot_ablation_centering(packs: List[DatasetPack], outdir="out"):
    """
    ablation_centering.png
    Bars = Spearman corr of (median/mean) vs (midmean baseline).
    Line = runtime ratio vs midmean (≈1 is good).
    """
    ensure_out(outdir)
    names, sp_med, sp_mean, rt_med, rt_mean = [], [], [], [], []
    for pack in packs:
        # use model to get p_va once
        base = train_xgb(pack.task, pack.Xtr, pack.ytr, pack.Xva, pack.yva, seed=GLOBAL_SEED)
        p_va = (base.predict_proba(pack.Xva.astype(np.float32))[:,1] if (pack.task=="clf" and hasattr(base,"predict_proba"))
                else base.predict(pack.Xva.astype(np.float32)))
        t0 = time.time(); s_mid = excir_scores(pack.Xva, p_va, center="midmean"); t_mid = time.time()-t0
        t0 = time.time(); s_med = excir_scores(pack.Xva, p_va, center="median");  t_med = time.time()-t0
        t0 = time.time(); s_mean= excir_scores(pack.Xva, p_va, center="mean");    t_mean= time.time()-t0
        names.append(pack.name)
        sp_med.append(spearman_corr(s_mid, s_med)); sp_mean.append(spearman_corr(s_mid, s_mean))
        rt_med.append((t_med / max(t_mid,1e-9)));   rt_mean.append((t_mean/ max(t_mid,1e-9)))
        del base; gc.collect()

    # Plot compactly
    plt.figure(figsize=(7.0,4.2))
    x = np.arange(len(names))
    width=0.35
    plt.bar(x - width/2, sp_med, width, label='Spearman(median, midmean)')
    plt.bar(x + width/2, sp_mean, width, label='Spearman(mean, midmean)')
    # runtime (as line over mean)
    plt.plot(x, rt_med, marker='o', label='Runtime ratio (median/midmean)')
    plt.plot(x, rt_mean, marker='o', label='Runtime ratio (mean/midmean)')
    plt.xticks(x, [n[:12] for n in names], rotation=45, ha='right')
    plt.ylim(0.0, 1.1); plt.legend(ncol=2); plt.tight_layout()
    plt.title("Centering choice ablation")
    plt.savefig(os.path.join(outdir, "ablation_centering.png"), dpi=200)
    plt.close()

def plot_ablation_blockcir_geneexpr(outdir="out"):
    """
    ablation_blockcir_geneexpr.png
    Left: score profiles (CIR vs BlockCIR) after sorting by CIR rank.
    Right: Jaccard@k overlap across k.
    """
    ensure_out(outdir)
    pack = ALL_DATASETS["gene_expression"](seed=GLOBAL_SEED)
    base = train_xgb(pack.task, pack.Xtr, pack.ytr, pack.Xva, pack.yva, seed=GLOBAL_SEED)
    p_va = (base.predict_proba(pack.Xva.astype(np.float32))[:,1] if hasattr(base, "predict_proba")
            else base.predict(pack.Xva.astype(np.float32)))
    s_cir = excir_scores(pack.Xva, p_va, center="midmean")
    order = np.argsort(-s_cir)
    # auto-blocks from val set
    blocks = auto_corr_blocks(pack.Xva, max_groups=60, corr_thresh=0.9)
    s_block = blockcir_scores(pack.Xva, p_va, blocks, center="midmean")

    # Expand block scores to feature order (for visual comparison)
    block_to_feat = np.zeros_like(s_cir)
    for bi, G in enumerate(blocks):
        for j in G: block_to_feat[j] = s_block[bi]
    s_block_feat_ordered = block_to_feat[order]
    s_cir_ordered = s_cir[order]

    # Jaccard@k overlap CIR vs BlockCIR-projected
    ks = np.arange(2, 61, 2)
    jac = []
    for k in ks:
        A = set(order[:k])
        B = set(np.argsort(-block_to_feat)[:k])
        jac.append(len(A & B)/float(len(A | B)))

    fig, ax = plt.subplots(1,2,figsize=(10,4.0))
    ax[0].plot(s_cir_ordered, label='CIR (feature-level)')
    ax[0].plot(s_block_feat_ordered, label='BlockCIR (expanded to features)')
    ax[0].set_title('Score profiles (sorted by CIR rank)'); ax[0].legend(); ax[0].grid(alpha=0.3)
    ax[1].plot(ks, jac, marker='o'); ax[1].set_title('Top-k Jaccard: CIR vs BlockCIR'); ax[1].set_xlabel('k'); ax[1].set_ylabel('Jaccard'); ax[1].grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, "ablation_blockcir_geneexpr.png"), dpi=200)
    plt.close()

def plot_pareto_mini_grid(packs: List[DatasetPack], outdir="out"):
    """
    pareto_mini_grid.png
    Each point: (runtime, Jaccard@8) for f in {0.2,0.3,0.4,0.6,0.8,1.0}, small multi-dataset grid.
    """
    ensure_out(outdir)
    figs = []
    plt.figure(figsize=(7.2,4.6))
    colors = plt.cm.tab10.colors
    for i, pack in enumerate(packs):
        # fit base once
        base = train_xgb(pack.task, pack.Xtr, pack.ytr, pack.Xva, pack.yva, seed=GLOBAL_SEED)
        p_va = (base.predict_proba(pack.Xva.astype(np.float32))[:,1] if (pack.task=="clf" and hasattr(base,"predict_proba"))
                else base.predict(pack.Xva.astype(np.float32)))
        s_full = excir_scores(pack.Xva, p_va, center="midmean")

        Xbig = np.vstack([pack.Xtr, pack.Xva]); ybig = np.r_[pack.ytr, pack.yva]
        fractions = (0.2, 0.3, 0.4, 0.6, 0.8, 1.0)
        T, J8 = [], []
        for f in fractions:
            t0=time.time()
            rs = check_random_state(GLOBAL_SEED)
            idx = rs.choice(np.arange(Xbig.shape[0]), size=max(50,int(f*Xbig.shape[0])), replace=False)
            params = dict(n_estimators=30, random_state=GLOBAL_SEED, n_jobs=1,
                          tree_method="gpu_hist", gpu_id=0, validate_parameters=True)
            m = _get_xgb_model(pack.task, ybig, params)
            try:
                m.fit(Xbig[idx].astype(np.float32), ybig[idx])
            except xgb.core.XGBoostError:
                params["tree_method"]="hist"; params.pop("gpu_id", None)
                m = _get_xgb_model(pack.task, ybig, params); m.fit(Xbig[idx].astype(np.float32), ybig[idx])
            p_sub = (m.predict_proba(pack.Xva.astype(np.float32))[:,1] if (pack.task=="clf" and hasattr(m,"predict_proba"))
                     else m.predict(pack.Xva.astype(np.float32)))
            s_sub = excir_scores(pack.Xva, p_sub, center="midmean")
            T.append(time.time()-t0); J8.append(jaccard_topk(s_full, s_sub, k=min(8, s_full.shape[0])))

        plt.plot(T, J8, marker='o', label=pack.name, color=colors[i % len(colors)], alpha=0.8)
        del base; gc.collect()

    plt.axhline(0.9, ls='--', lw=1.2, alpha=0.6)  # target overlap line
    plt.xlabel("Wall time (s)"); plt.ylabel("Jaccard@8 (full vs light)")
    plt.title("Lightweight fraction sweep (mini Pareto)")
    plt.legend(ncol=2, fontsize=8)
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, "pareto_mini_grid.png"), dpi=200)
    plt.close()

# ==========================
# 7) MASTER EXECUTION
# ==========================
def run_all(
    datasets: Optional[List[str]] = None,
    outdir: str = "out",
    do_pfi: bool = True
):
    ensure_out(outdir)
    names = datasets if datasets else list(ALL_DATASETS.keys())
    print(f"--- ExCIR Benchmark on {len(names)} datasets ---")
    for name in names:
        try:
            pack = ALL_DATASETS[name](seed=GLOBAL_SEED)
            run_dataset(pack, ks=(3,5,8,12), center="midmean",
                        outdir=outdir, do_pfi=do_pfi, do_blockcir=True)
        except Exception as e:
            print(f"[ERROR] {name}: {e}")
            import traceback; traceback.print_exc()

    # Global plots for the paper
    # Centering ablation: choose 3 diverse datasets
    subset_for_centering = [
        ALL_DATASETS["california"](seed=GLOBAL_SEED),
        ALL_DATASETS["digits8x8"](seed=GLOBAL_SEED),
        ALL_DATASETS["network_topology"](seed=GLOBAL_SEED)
    ]
    plot_ablation_centering(subset_for_centering, outdir=outdir)

    # BlockCIR ablation on gene_expression
    plot_ablation_blockcir_geneexpr(outdir=outdir)

    # Pareto mini-grid on 4 diverse datasets (fast)
    subset_for_pareto = [
        ALL_DATASETS["har6"](seed=GLOBAL_SEED),
        ALL_DATASETS["fashion_mnist_features"](seed=GLOBAL_SEED),
        ALL_DATASETS["california"](seed=GLOBAL_SEED),
        ALL_DATASETS["stock_technical"](seed=GLOBAL_SEED),
    ]
    plot_pareto_mini_grid(subset_for_pareto, outdir=outdir)



In [7]:
# ==========================
# 8) ENTRY POINT (Notebook)
# ==========================

# ---- User toggles ----
RUN_ALL_29 = True        # True = run all 29 datasets; False = use SHORT_LIST below
USE_PFI    = False       # True = include PFI (slower); False = skip PFI (faster)
OUTDIR     = "out"       # output directory
SHORT_LIST = ["iris", "diabetes", "digits8x8"]  # used only if RUN_ALL_29=False
# ----------------------

if RUN_ALL_29:
    print("Running ALL 29 datasets...")
    run_all(datasets=None, outdir=OUTDIR, do_pfi=USE_PFI)
else:
    print(f"Running SHORT LIST: {SHORT_LIST}")
    run_all(datasets=SHORT_LIST, outdir=OUTDIR, do_pfi=USE_PFI)

print("\nDone. Artifacts written to:", OUTDIR)


Running ALL 29 datasets...
--- ExCIR Benchmark on 29 datasets ---
[adult] N=48842  D=6  task=clf
[20ng_bin] N=1960  D=3000  task=clf
[har6] N=10299  D=561  task=clf
[digits8x8] N=1797  D=64  task=clf
[california] N=20640  D=8  task=reg
[diabetes] N=442  D=10  task=reg
[iris] N=150  D=4  task=clf
[wine] N=178  D=13  task=clf
[synthetic_clf] N=3000  D=40  task=clf
[synthetic_reg] N=3000  D=40  task=reg
[ecg_heartbeat] N=2500  D=8  task=clf
[fashion_mnist_features] N=5000  D=128  task=clf
[ERROR] protein_structure: too many positional arguments
[stock_technical] N=4000  D=5  task=clf


Traceback (most recent call last):
  File "/tmp/ipython-input-631934397.py", line 912, in run_all
    pack = ALL_DATASETS[name](seed=GLOBAL_SEED)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-631934397.py", line 491, in load_protein_structure
    def load_protein_structure(seed=0): X,y=make_regression(4000,120,20,noise=2.0,random_state=seed); return _split_pack_from_xy(X,y,"protein_structure","reg",seed)
                                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/_param_validation.py", line 194, in wrapper
    params = func_sig.bind(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/inspect.py", line 3280, in bind
    return self._bind(args, kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/inspect.py", line 3204, in _bind
    raise TypeError(
TypeError: too many positional arguments


[sensor_fusion] N=3000  D=24  task=clf
[gene_expression] N=800  D=2000  task=clf
[network_topology] N=3000  D=5  task=clf
[audio_mfcc] N=3500  D=20  task=clf
[weather_station] N=5000  D=5  task=reg
[eeg_signals] N=3000  D=32  task=clf
[satellite_ndvi] N=4000  D=8  task=reg
[industrial_process] N=4500  D=4  task=reg
[social_network] N=5000  D=5  task=clf
[ERROR] cyber_security: too many positional arguments
[ERROR] mixed_modal: too many positional arguments
[breast_cancer] N=569  D=30  task=clf


Traceback (most recent call last):
  File "/tmp/ipython-input-631934397.py", line 912, in run_all
    pack = ALL_DATASETS[name](seed=GLOBAL_SEED)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-631934397.py", line 536, in load_cyber_security
    X,y=make_classification(5000,30,8,10,weights=[0.8,0.2],class_sep=1.5,random_state=seed)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/_param_validation.py", line 194, in wrapper
    params = func_sig.bind(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/inspect.py", line 3280, in bind
    return self._bind(args, kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/inspect.py", line 3204, in _bind
    raise TypeError(
TypeError: too many positional arguments
Traceback (most recent call last):
  File "/tmp/ipython-input-631934397.py", line 912, in run_all


[retail_demand] N=5000  D=4  task=reg
[traffic_flow] N=4500  D=3  task=reg
[energy_consumption] N=4000  D=4  task=reg

Done. Artifacts written to: out


In [ ]:
ALL_DATASETS = {
    "adult":load_adult,"20ng_bin":load_20ng_binary,"har6":load_har,"digits8x8":load_digits8x8,
    "california":load_california,"diabetes":load_diabetes,"iris":load_iris,"wine":load_wine,
    "synthetic_clf":load_synthetic_clf,"synthetic_reg":load_synthetic_reg,
    "ecg_heartbeat":load_ecg_heartbeat,"fashion_mnist_features":load_fashion_mnist_features,
    "protein_structure":load_protein_structure,"stock_technical":load_stock_technical,
    "sensor_fusion":load_sensor_fusion,"gene_expression":load_gene_expression,
    "network_topology":load_network_topology,"audio_mfcc":load_audio_mfcc,
    "weather_station":load_weather_station,"eeg_signals":load_eeg_signals,
    "satellite_ndvi":load_satellite_ndvi,"industrial_process":load_industrial_process,
    "social_network":load_social_network,"cyber_security":load_cyber_security,"mixed_modal":load_mixed_modal,
}